# Trovare le "Golden Labels"

Questo notebook confronta due dataset per identificare le "golden labels".

1.  **Dataset EmoSign (Hugging Face)**: Contiene i nomi dei video con un ID utterance.
2.  **Dataset ASLLRP Sentiment**: Il nostro dataset locale con analisi del sentiment, generato in precedenza.

L'obiettivo è trovare le righe nel nostro dataset locale che hanno un ID utterance corrispondente nel dataset EmoSign. Queste righe verranno considerate "golden labels" e salvate in un nuovo file.


In [13]:
# Step 2: Caricare i dataset e preparare gli ID
import pandas as pd

# --- Caricamento e preparazione del dataset EmoSign da Hugging Face ---
print("Caricamento del dataset EmoSign da Hugging Face...")
try:
    # Per accedere a questo dataset, potrebbe essere necessario un login tramite `huggingface-cli login`
    df_emosign = pd.read_csv("hf://datasets/catfang/emosign/emosign_dataset.csv")
    print("Dataset EmoSign caricato con successo.")

    # Estrai l'ID utterance dalla colonna 'video_name'
    # L'ID è la parte numerica dopo il trattino basso '_'
    df_emosign["utterance_id"] = (
        df_emosign["video_name"].str.split("_").str[-1].astype(str)
    )
    emosign_ids = set(df_emosign["utterance_id"])
    print(f"Trovati {len(emosign_ids)} ID unici nel dataset EmoSign.")

except Exception as e:
    print(f"Errore durante il caricamento del dataset da Hugging Face: {e}")
    print(
        "Assicurati di aver effettuato l'accesso con 'huggingface-cli login' nel tuo terminale."
    )
    emosign_ids = set()

# --- Caricamento e preparazione del dataset locale ---
if emosign_ids:
    print("\nCaricamento del dataset locale con sentiment...")
    local_csv_path = "../data/processed/asllrp_video_sentiment_data_0.34.csv"
    try:
        df_local = pd.read_csv(local_csv_path)
        print("Dataset locale caricato con successo.")

        # Estrai l'ID utterance dalla colonna 'Utterance video filename'
        # Rimuovi l'estensione '.mp4'
        df_local["utterance_id"] = (
            df_local["Utterance video filename"]
            .str.replace(".mp4", "", regex=False)
            .astype(str)
        )
        local_ids = set(df_local["utterance_id"])
        print(f"Trovati {len(local_ids)} ID unici nel dataset locale.")

    except FileNotFoundError:
        print(f"Errore: File non trovato a '{local_csv_path}'")
        df_local = pd.DataFrame()
else:
    print("\nSalto il caricamento del dataset locale a causa di errori precedenti.")
    df_local = pd.DataFrame()

Caricamento del dataset EmoSign da Hugging Face...
Dataset EmoSign caricato con successo.
Trovati 200 ID unici nel dataset EmoSign.

Caricamento del dataset locale con sentiment...
Dataset locale caricato con successo.
Trovati 830 ID unici nel dataset locale.
Dataset EmoSign caricato con successo.
Trovati 200 ID unici nel dataset EmoSign.

Caricamento del dataset locale con sentiment...
Dataset locale caricato con successo.
Trovati 830 ID unici nel dataset locale.


In [14]:
# Step 3: Trovare le corrispondenze ("Golden Labels")
if "df_local" in locals() and not df_local.empty and emosign_ids:
    # Trova gli ID in comune tra i due dataset
    golden_ids = local_ids.intersection(emosign_ids)
    print(f"\nTrovate {len(golden_ids)} corrispondenze ('golden labels').")

    # Filtra il DataFrame locale per mantenere solo le righe con gli ID corrispondenti
    golden_labels_df = df_local[df_local["utterance_id"].isin(golden_ids)].copy()

    # Rimuovi la colonna 'utterance_id' di supporto se non più necessaria
    golden_labels_df.drop(columns=["utterance_id"], inplace=True)

    print("\nPrime 5 righe delle 'golden labels':")
    print(golden_labels_df.head())
else:
    print(
        "\nImpossibile trovare le corrispondenze a causa di errori nel caricamento dei dati."
    )
    golden_labels_df = pd.DataFrame()


Trovate 200 corrispondenze ('golden labels').

Prime 5 righe delle 'golden labels':
  Utterance video filename                                        translation  \
4             83664512.mp4  I was like, “Oh, wow, that`s fine.” He wanted ...   
6                25328.mp4  There are so many things that are wrong with t...   
7               253715.mp4  Then they would ask the HIT group about the ca...   
8               255438.mp4  Or they will ask another group SMASHED about t...   
9               253816.mp4  Or ask the COLLIDED group about the car and if...   

    emotion  
4  Positive  
6  Negative  
7  Negative  
8  Negative  
9  Negative  


In [15]:
# Step 4: Salvare il nuovo DataFrame
if not golden_labels_df.empty:
    output_path = "../data/processed/golden_label_sentiment.csv"
    golden_labels_df.to_csv(output_path, index=False)
    print(
        f"\nFile 'golden_label_sentiment.csv' salvato con successo in '{output_path}'."
    )
    print(f"Il file contiene {len(golden_labels_df)} righe.")
else:
    print("\nNessun dato da salvare.")


File 'golden_label_sentiment.csv' salvato con successo in '../data/processed/golden_label_sentiment.csv'.
Il file contiene 200 righe.


In [16]:
# Step 5: Salvare il dataset locale escludendo le "Golden Labels"
if "df_local" in locals() and not df_local.empty and "golden_ids" in locals():
    # Filtra il DataFrame locale per escludere le righe con gli ID "golden"
    df_without_golden = df_local[~df_local["utterance_id"].isin(golden_ids)].copy()

    # Rimuovi la colonna 'utterance_id' di supporto
    df_without_golden.drop(columns=["utterance_id"], inplace=True)

    # Definisci il percorso del nuovo file CSV
    # Prende il nome del file originale e aggiunge "_without_golden"
    original_path = "../data/processed/asllrp_video_sentiment_data_0.34.csv"
    base, ext = os.path.splitext(original_path)
    output_path_without_golden = f"{base}_without_golden{ext}"

    # Salva il nuovo DataFrame
    df_without_golden.to_csv(output_path_without_golden, index=False)

    print(
        f"\nFile '{os.path.basename(output_path_without_golden)}' salvato con successo."
    )
    print(f"Contiene {len(df_without_golden)} righe (righe originali - golden labels).")
    print(
        f"Righe originali: {len(df_local)}, Golden Labels: {len(golden_ids)}, Righe rimanenti: {len(df_without_golden)}"
    )

else:
    print(
        "\nImpossibile creare il file senza golden labels a causa di errori precedenti o dati mancanti."
    )


File 'asllrp_video_sentiment_data_0.34_without_golden.csv' salvato con successo.
Contiene 630 righe (righe originali - golden labels).
Righe originali: 830, Golden Labels: 200, Righe rimanenti: 630
